# Deploying apps for on-prem and hybrid ML+AI 
# on SQL Server 2019 Big Data Clusters
## //build 2019
Anna Thomas, @AnalyticAnna  
Jeroen ter Heerdt, @jeroenterheerdt  

> Note: This notebook assumes that you have completed the setup requirements in **setup.md**.


## Enabling ML Services + Extensibility on the SQL Server Big Data Cluster master instance

In [1]:
sp_configure 'external scripts enabled', 1;
RECONFIGURE WITH OVERRIDE;  

Configuration option 'external scripts enabled' changed from 1 to 1. Run the RECONFIGURE statement to install.

Total execution time: 00:00:00.1700496

## Use the WideWorldImporters sample

In [2]:
USE WideWorldImporters;

Commands completed successfully.

Total execution time: 00:00:00.0340901

## Create an external library to reference the Java Extension sdk

In [3]:
CREATE EXTERNAL LIBRARY sdk
FROM (CONTENT = '/home/mssql_java_lang_extension.jar')
WITH (LANGUAGE = 'Java');

: Msg 39047, Level 16, State 1, Line 1
External library 'sdk' already exists for owner 'dbo' in database 'WideWorldImporters'.

Total execution time: 00:00:00.0376209

## Regex in Java
An example of how to use Java for something that's just easier in Java than TSQL

In [3]:
DROP TABLE IF exists pii_numbers

CREATE TABLE pii_numbers(
	id int NOT NULL,
	"text" nvarchar(60) NOT NULL)
    Truncate table pii_numbers

INSERT INTO pii_numbers(id, "text") VALUES (1, 'here it is 123-45-6789')
INSERT INTO pii_numbers(id, "text") VALUES (2, 'the id for the purchase was 9876-5-4321')
INSERT INTO pii_numbers(id, "text") VALUES (3, 'you can call 965-444-3045 for more info')
INSERT INTO pii_numbers(id, "text") VALUES (4, 'id 192-83-7465 was taken in and processed');

select * from pii_numbers

(1 row affected)

(1 row affected)

(1 row affected)

(1 row affected)

(4 rows affected)

Total execution time: 00:00:00.3087131

id,text
1,here it is 123-45-6789
2,the id for the purchase was 9876-5-4321
3,you can call 965-444-3045 for more info
4,id 192-83-7465 was taken in and processed


In [4]:
DROP EXTERNAL LIBRARY regex
CREATE EXTERNAL LIBRARY regex
FROM (CONTENT = '/home/regex.jar') 
WITH (LANGUAGE = 'Java'); 

: Msg 15151, Level 16, State 2, Line 1
Cannot drop the external library 'regex', because it does not exist or you do not have permission.

Total execution time: 00:00:00.2396356

In [5]:
DECLARE @expression nvarchar(100)
SET @expression = N'[0-9]{3}?-[0-9]{2}-?[0-9]{4}'
EXEC sp_execute_external_script
  @language = N'Java'
, @script = N'pkg.regexSample'
, @input_data_1 = N'SELECT * FROM pii_numbers'
, @params = N'@regexExpr nvarchar(40)'
, @regexExpr = @expression
with result sets ((ID int, text nvarchar(60)))

(2 rows affected)

Total execution time: 00:00:15.6462056

ID,text
1,here it is 123-45-6789
4,id 192-83-7465 was taken in and processed


## Sentiment Analysis in Java
Here, we're leveraging the Text Analytics API from Azure Cognitive Services to get the sentiment of product reviews.

In [6]:
USE WideWorldImporters;

SELECT TOP(25) productId, review 
FROM dbo.product_reviews

(25 rows affected)

Total execution time: 00:00:00.0740858

productId,review
17759,great product to save money! Dont worry about leaving the light on anymore. It is great for kitchen! My son can help me season our food with out making mess and this fits just fine in the hand and it never dulled; rusted; or got out of shape. Perfect quality and very easy and effortless to use. This blade is ideal for both narrow and wide wedges. The curve at the local Home Depot store. Both seem to work with. In my case fan). It's usually pretty easy to determine which cable is hot (that being said it's always best to check using volt meter between what you think is hot (that being said it's always best to check using volt meter between what you think is hot and the ground wire you obviously should drop power to the OXO the overall build of the other &quot;Waterless&quot; drink coolers that we've had since long before the grated food has seal to prevent leaking while shaking your favorite drink.
9955,Next time will go with the old metal handle- this is bonus.
16529,Great Gift Great Value had to get used. And after 12 hours of use; they just throw them away; so you haven't created any useless clutter. (Get yourself set too.)
1019,After trip to Paris and falling in love with Nutella crepes decided had to try it. am glad found it! Thank you; CIA; for my existing switch. Design-wise it is dishwasher safe too! Very highly recommended. You'll thank me for this!JANA
2260,Simply the best thing about them is that you can only use for one thing; so this one is wonderful to hold the keys.
3592,If you enjoy spy novels primarily for their bastard pop tracks; and they make me want to get if you want to be; in the UK and will be right at home on your shelf. That's right its an album! That means the whole EP.....Ooh La La from Supernature; Goldfrapp returns with #1 and what masterpiece of an album really is. highly recommend Lietmotif and El Cielo at the advice of the 70's when read that the music is really great so hopefully one will be able to enjoy; but atleast the few records know that remember now!!
11504,after having apple power cord. Ive had this thing for months now and its perfect; not even be here looking or buying these at all and the Macally PS-AC4 that Amazon has is the lack of charging indicator light. This is brilliant white. The cords are sturdy and stays connected to the original Macally product. The Macally adapter is bargain compared to the product that looked nothing like the mac one to tell you if your worried about the same shape but the price is much better than the mac adaptor. The size is manageable. While it doesn't do much else. So if your original cord is more functional in airports and hotel rooms. The cord has version that is pictured.It works perfectly fine with my Apple charger split and threw sparks and flames into the unit. The unit provides solid grounding and protection for you. Lights illuminate to show you that the unit that better get replacement from Apple. And if your original plug is still plugging away; my advice would be to purchase the Macally one. have to carry them back and forth. The one at home (Macally). After doing some research and reading lots of reviews; decided on large project with flammables everywhere filed reports; and Apple is acutely aware of the Macally immediately and seems lot nicer looking in person it is highly regarded company in the NW US). It's working fine -- unlike some other reviews I've read; I've found it to be contributing factor to all of my laptops or netook in without using this. They are truly the Achilles heal of the others who have written reviews; the pin for my laptop between rooms. And prefer this one sitting at home and leave the apartment. small win; but well worth the price and equal in every way.
11880,This is the exact product that my mother used in the outlet/switch box. It does exactly what was glad to find so was happy to finally get them. great service. thank you.
3219,After the wiring on my Apple charger quit working years ago

In [7]:
DROP EXTERNAL LIBRARY sentiment
DROP EXTERNAL LIBRARY luis
DROP EXTERNAL LIBRARY regex

CREATE EXTERNAL LIBRARY sentiment
FROM (CONTENT = '/home/sentiment.jar') 
WITH (LANGUAGE = 'Java'); 

: Msg 15151, Level 16, State 2, Line 1
Cannot drop the external library 'sentiment', because it does not exist or you do not have permission.

Total execution time: 00:00:00.2521864

In [8]:
EXEC sp_execute_external_script
  @language = N'Java'
, @script=N'pkg.sentimentSample'
, @input_data_1 = N'SELECT TOP(25) convert(int, productId), review FROM dbo.product_reviews'
, @parallel = 0
with result sets ((ID int, review nvarchar(max), sentimentScore nvarchar(5)))

(25 rows affected)

Total execution time: 00:00:18.3846353

ID,review,sentimentScore
17759,great product to save money! Dont worry about leaving the light on anymore. It is great for kitchen! My son can help me season our food with out making mess and this fits just fine in the hand and it never dulled; rusted; or got out of shape. Perfect quality and very easy and effortless to use. This blade is ideal for both narrow and wide wedges. The curve at the local Home Depot store. Both seem to work with. In my case fan). It's usually pretty easy to determine which cable is hot (that being said it's always best to check using volt meter between what you think is hot (that being said it's always best to check using volt meter between what you think is hot and the ground wire you obviously should drop power to the OXO the overall build of the other &quot;Waterless&quot; drink coolers that we've had since long before the grated food has seal to prevent leaking while shaking your favorite drink.,0.874
9955,Next time will go with the old metal handle- this is bonus.,0.777
16529,Great Gift Great Value had to get used. And after 12 hours of use; they just throw them away; so you haven't created any useless clutter. (Get yourself set too.),0.146
1019,After trip to Paris and falling in love with Nutella crepes decided had to try it. am glad found it! Thank you; CIA; for my existing switch. Design-wise it is dishwasher safe too! Very highly recommended. You'll thank me for this!JANA,0.981
2260,Simply the best thing about them is that you can only use for one thing; so this one is wonderful to hold the keys.,0.895
3592,If you enjoy spy novels primarily for their bastard pop tracks; and they make me want to get if you want to be; in the UK and will be right at home on your shelf. That's right its an album! That means the whole EP.....Ooh La La from Supernature; Goldfrapp returns with #1 and what masterpiece of an album really is. highly recommend Lietmotif and El Cielo at the advice of the 70's when read that the music is really great so hopefully one will be able to enjoy; but atleast the few records know that remember now!!,0.931
11504,after having apple power cord. Ive had this thing for months now and its perfect; not even be here looking or buying these at all and the Macally PS-AC4 that Amazon has is the lack of charging indicator light. This is brilliant white. The cords are sturdy and stays connected to the original Macally product. The Macally adapter is bargain compared to the product that looked nothing like the mac one to tell you if your worried about the same shape but the price is much better than the mac adaptor. The size is manageable. While it doesn't do much else. So if your original cord is more functional in airports and hotel rooms. The cord has version that is pictured.It works perfectly fine with my Apple charger split and threw sparks and flames into the unit. The unit provides solid grounding and protection for you. Lights illuminate to show you that the unit that better get replacement from Apple. And if your original plug is still plugging away; my advice would be to purchase the Macally one. have to carry them back and forth. The one at home (Macally). After doing some research and reading lots of reviews; decided on large project with flammables everywhere filed reports; and Apple is acutely aware of the Macally immediately and seems lot nicer looking in person it is highly regarded company in the NW US). It's working fine -- unlike some other reviews I've read; I've found it to be contributing factor to all of my laptops or netook in without using this. They are truly the Achilles heal of the others who have written reviews; the pin for my laptop between rooms. And prefer this one sitting at home and leave the apartment. small win; but well worth the price and equal in every way.,0.900
11880,This is the exact product that my mother used in the outlet/switch box. It does exactly what was glad to find so was happy to finally get them. great service. thank you.,0.991
3219,Aft

## Natural Language Understanding in Java

Here, we used Azure Cognitive Services, specifically LUIS, to create a custom language understanding model so we can identify intents and entities within free text, like delivery instructions. 

In [9]:
USE WideWorldImporters;

SELECT TOP(10) convert(int, OrderID) as ID, DeliveryInstructions 
FROM dbo.Deliveries

(10 rows affected)

Total execution time: 00:00:00.0444724

ID,DeliveryInstructions
1,headed to houston from Detroit Sunday 12 PM
2,from: new york. To: atlanta Day: Monday 3PM
3,Arriving from new york in seattle on Thurs at noon
4,Seattle to san francisco tues 5pm
5,"going to detroit from los angeles, on monday at 9am"
6,Deliver from seattle location to Detroit on saturday at 8PM.
7,"Faced delays, arriving 1 pm Friday from Detroit. Destination atlanta"
8,Destination is miami at 4pm Sunday. Beginning in chicago
9,"Leaving and arriving monday at 11:30AM, miami to atlanta"
10,"going from San Francisco to chicago, Saturday 10 am"


In [10]:
DROP EXTERNAL LIBRARY luis
DROP EXTERNAL LIBRARY regex
DROP EXTERNAL LIBRARY sentiment

CREATE EXTERNAL LIBRARY luis
FROM (CONTENT = '/home/luis.jar') 
WITH (LANGUAGE = 'Java'); 

: Msg 15151, Level 16, State 2, Line 1
Cannot drop the external library 'luis', because it does not exist or you do not have permission.

: Msg 15151, Level 16, State 2, Line 2
Cannot drop the external library 'regex', because it does not exist or you do not have permission.

Total execution time: 00:00:00.5570333

In [11]:
EXEC sp_execute_external_script
  @language = N'Java'
, @script=N'pkg.luisSample'
, @input_data_1 = N'SELECT TOP(10) convert(int, OrderID) as ID, DeliveryInstructions 
FROM dbo.Deliveries'
, @parallel = 0
with result sets ((ID int, DeliveryInstructions nvarchar(max), Origin nvarchar(50), Destination nvarchar(50), DeliveryDateTime nvarchar(50)))

(10 rows affected)

Total execution time: 00:00:18.4759690

ID,DeliveryInstructions,Origin,Destination,DeliveryDateTime
1,headed to houston from Detroit Sunday 12 PM,detroit,houston,2019-05-05 12:00:00
2,from: new york. To: atlanta Day: Monday 3PM,new york,atlanta,2019-04-29 15:00:00
3,Arriving from new york in seattle on Thurs at noon,new york,seattle,2019-05-02 12:00:00
4,Seattle to san francisco tues 5pm,seattle,san francisco,2019-04-30 17:00:00
5,"going to detroit from los angeles, on monday at 9am",los angeles,detroit,2019-04-29 09:00:00
6,Deliver from seattle location to Detroit on saturday at 8PM.,seattle,detroit,2019-05-04 20:00:00
7,"Faced delays, arriving 1 pm Friday from Detroit. Destination atlanta",detroit,atlanta,2019-05-03 13:00:00
8,Destination is miami at 4pm Sunday. Beginning in chicago,chicago,miami,2019-05-05 16:00:00
9,"Leaving and arriving monday at 11:30AM, miami to atlanta",miami,atlanta,2019-04-29 11:30:00
10,"going from San Francisco to chicago, Saturday 10 am",chicago,san francisco,2019-05-04 10:00:00


# Anyone see any potential issues with this?



.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
.  
.  

    
      

    
      

    
      


## DataOps

In order to deploy the LUIS container into the cluster, we created a yaml file (which you personalized in the setup for this) to make it easier. In the terminal (not Azure Data Studio), run through the following to deploy and run the container in the big data cluster.  

Deploy the container in the cluster  
`kubectl apply -f /home/luis-cognitive-service.yaml -n <clusterName>`

Inspect loadbalancer IP for the next command  
`kubectl describe svc cognitive-luis-svc`  

Use the loadbalancer ingress IP from above to replace the IP below, and add your LUIS Application ID to test the container  
`curl "http://loadbalancerIngressIpHere:5000/luis/v2.0/apps/YourAppIdHere?q=from%20atlanta%20to%20seattle%20tuesday%20at%204PM&staging=false&timezoneOffset=0&verbose=false&log=true" -H "accept: application/json"`  

Finally, you need to update the LUIS Java program in eclipse to point "host" to the loadbalancer ingress IP instead of "https://centralus.api.cognitive.microsoft.com". You'll have to recompile and export the jar, and then move it into the cluster. Refer to the setup document for more details.

## Update the Java program

In [0]:
DROP EXTERNAL LIBRARY luis

CREATE EXTERNAL LIBRARY luis
FROM (CONTENT = '/home/luis-container.jar') 
WITH (LANGUAGE = 'Java');

In [0]:
EXEC sp_execute_external_script
  @language = N'Java'
, @script=N'pkg.luisSample'
, @input_data_1 = N'SELECT TOP(10) convert(int, OrderID) as ID, DeliveryInstructions 
FROM dbo.Deliveries'
, @parallel = 0
with result sets ((ID int, DeliveryInstructions nvarchar(max), Origin nvarchar(50), Destination nvarchar(50), DeliveryDateTime nvarchar(50)))